<a href="https://colab.research.google.com/github/mukul-mschauhan/GenerativeAI/blob/main/React_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Agents
Create a Google Colab notebook that implements an AI agent using the ReACT (Reason+Act) paradigm. This notebook should include:
1.  **Introduction**: Markdown cells explaining AI agents, ReACT, the role of tools (specifically Tavily for web search), and the concept of a 'Decision Trace'.
2.  **Setup**: Code cells for installing necessary Python packages (`openai`, `tavily-python`, `gradio`) and securely setting `OPENAI_API_KEY` and `TAVILY_API_KEY` environment variables with robust error handling.
3.  **Tavily Search Tool**: A Python function `tavily_search` to interact with the Tavily API, returning structured search results (title, URL, content/snippet) with error handling.
4.  **Search Gating Logic**: A Python function `should_use_tavily` that determines whether to use web search based on the user's query (e.g., for recency, factual lookups, keywords like 'latest', 'verify'), providing a boolean decision and a reason.
5.  **ReACT Agent Core**: A Python function (e.g., `run_agent`) that serves as the core agent logic. It should take a user query, use `should_use_tavily` to decide on web search, execute `tavily_search` if needed, and then use the OpenAI API to synthesize a final answer. This function must construct a 'Decision Trace' and a 'Tool Trace'.
6.  **Gradio Interactive Frontend**: A Gradio interface with a textbox for user input, a 'Mode' dropdown ('Teach Mode' to show all traces, 'Normal Mode' to show only the final answer), and a 'Clear' button. The UI should display 'User question', 'Decision Trace', 'Tool Trace', and 'Final Answer'. Preload example questions like 'What is a ReACT agent? Explain with audit analogy.', 'What are the latest UAE VAT updates?', and 'Create an audit evidence request list for revenue testing.'. The app should run in Colab with `share=False`.
7.  **Output Formatting**: Ensure the agent's output is consistently formatted into distinct sections: 'Decision Trace' (Search needed? Yes/No, Why?), 'Tool Trace' (search query + results if searched, otherwise 'No external tools used'), and 'Final Answer' (helpful, adopt a UAE audit/accounting tone, with citations if web search was used).
8.  **Educational Lab Exercise**: A markdown cell detailing a mini-lab exercise that prompts users to modify the `should_use_tavily` gating rules and observe the agent's behavior.
The notebook should be self-contained and ready for execution.

## Notebook Introduction and Setup

### Subtask:
Create a markdown cell explaining AI agents, ReACT, the role of tools (Tavily), and Decision Trace. Then, add code cells to install necessary Python packages and securely set API keys.


## Notebook Introduction and Setup

### Subtask:
Create a markdown cell explaining AI agents, ReACT, the role of tools (Tavily), and Decision Trace. Then, add code cells to install necessary Python packages and securely set API keys.

# AI Agent with ReACT Paradigm

### What is an AI agent?
An AI agent is a program designed to perceive its environment through sensors, process that information, and then act upon that environment through effectors to achieve specific goals. Unlike simple scripts, agents can make decisions, learn, and adapt to changing conditions. They often employ sophisticated reasoning mechanisms and can utilize various tools to extend their capabilities.

### What is ReACT (Reason+Act)?
ReACT stands for "Reason + Act". It's a paradigm for designing AI agents where the agent explicitly *reasons* about a problem, *plans* its next steps, and then *acts* based on that plan. This iterative process allows the agent to break down complex tasks into smaller, manageable subtasks, use tools effectively, and self-correct when faced with unexpected outcomes. It improves the agent's transparency, reliability, and capability to handle more intricate challenges.

### What is the role of tools (specifically Tavily for web search)?
Tools empower AI agents by extending their capabilities beyond their inherent knowledge. They allow agents to interact with the external world, retrieve up-to-date information, perform computations, or even control other software. Tavily, for instance, serves as a powerful web search tool. When an agent needs information not contained within its training data or requires real-time data, it can use Tavily to perform a web search, analyze the results, and incorporate that fresh information into its reasoning process, thus making more informed decisions.

### Why we show Decision Trace instead of private reasoning
In AI agent development, especially with complex models, the internal "thought process" or "private reasoning" can be very intricate and often difficult to interpret directly. A "Decision Trace" is a structured log of the agent's thought process, showcasing the key steps it takes: what it observes, what it reasons, what tool it decides to use, the input to that tool, and the tool's output, leading to its final action. Showing the decision trace helps in:
1.  **Transparency:** Understanding *how* the agent arrived at a particular conclusion or action.
2.  **Debugging:** Identifying where the agent's reasoning might have gone astray.
3.  **Trust:** Building user confidence by demonstrating a logical flow of operations.
4.  **Learning:** Providing insights into agent behavior for further refinement and improvement.

**Reasoning**:
The next step is to install the necessary Python packages as specified in the instructions.



In [1]:
pip install openai tavily-python gradio

**Reasoning**:
The next step is to securely set the API keys for OpenAI and Tavily, including robust error handling, as specified in the instructions. This ensures the agent can interact with these services.



In [2]:
import os
from getpass import getpass

def set_api_key(api_key_name):
    """Sets an API key from environment variable or user input."""
    if api_key_name not in os.environ:
        os.environ[api_key_name] = getpass(f"Enter your {api_key_name}:")

    if not os.environ.get(api_key_name):
        raise ValueError(f"{api_key_name} is not set. Please set the API key to proceed.")
    print(f"{api_key_name} is set.")

try:
    set_api_key("OPENAI_API_KEY")
    set_api_key("TAVILY_API_KEY")
    print("All necessary API keys are securely set.")
except ValueError as e:
    print(f"Error: {e}")
    print("Please ensure you enter valid API keys when prompted or set them as environment variables.")

Enter your OPENAI_API_KEY:··········
OPENAI_API_KEY is set.
Enter your TAVILY_API_KEY:··········
TAVILY_API_KEY is set.
All necessary API keys are securely set.


## Implement Tavily Search Tool

### Subtask:
Define the `tavily_search` function to interact with the Tavily API, returning structured search results with error handling.


**Reasoning**:
I need to define the `tavily_search` function as specified, which includes importing `TavilyClient`, instantiating it, handling potential errors, and structuring the search results.



In [3]:
import os
from tavily import TavilyClient

def tavily_search(query: str, max_results: int = 5) -> list:
    """
    Performs a web search using the Tavily API and returns structured results.

    Args:
        query (str): The search query.
        max_results (int): The maximum number of search results to return.

    Returns:
        list: A list of dictionaries, each containing 'title', 'url', and 'content' of a search result.
              Returns an empty list if an error occurs or no results are found.
    """
    try:
        tavily_client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))
        response = tavily_client.search(
            query=query,
            search_depth='advanced',
            max_results=max_results
        )

        structured_results = []
        for result in response.get('results', []):
            structured_results.append({
                'title': result.get('title'),
                'url': result.get('url'),
                'content': result.get('content')
            })
        return structured_results
    except Exception as e:
        print(f"An error occurred during Tavily search: {e}")
        return []

print("The `tavily_search` function is defined and ready for use.")

The `tavily_search` function is defined and ready for use.


## Implement Search Gating Logic

### Subtask:
Create the `should_use_tavily` function that determines whether to use web search based on the user's query, providing a boolean decision and a reason.


**Reasoning**:
I will now define the `should_use_tavily` function. This function is crucial for the agent's efficiency, as it prevents unnecessary API calls to Tavily for queries that can be answered from the agent's internal knowledge or are purely conceptual. The logic will involve checking for specific keywords or phrases that indicate a need for up-to-date information, factual verification, or external data, while also identifying queries that are generic or conceptual and do not require a web search. This function will return a boolean indicating whether to use Tavily and a string explaining the decision.

**Reasoning**:
Now I will implement the `should_use_tavily` function based on the detailed instructions provided, including keyword detection for web search necessity and identification of conceptual queries, ensuring case-insensitivity and returning a boolean and a reason.



In [4]:
def should_use_tavily(user_query: str) -> tuple[bool, str]:
    """
    Determines whether a web search using Tavily is necessary based on the user's query.

    Args:
        user_query (str): The user's input query.

    Returns:
        tuple[bool, str]: A tuple containing a boolean (True if Tavily should be used, False otherwise)
                         and a string explaining the decision.
    """
    use_tavily = False
    reason = "Query appears conceptual or does not contain keywords suggesting a need for web search."

    # Convert query to lowercase for case-insensitive checking
    query_lower = user_query.lower()

    # Keywords indicating a need for web search (recency, factual lookup)
    search_keywords = [
        'latest', 'today', 'news', 'current', 'recent', 'updates',
        'verify', 'what are the', 'who is the', 'when was', 'how many',
        'statistics', 'report', 'data', 'facts', 'figures', 'policy', 'regulation'
    ]

    # Phrases/patterns indicating conceptual or evergreen queries (less likely to need search)
    conceptual_phrases = [
        'what is a', 'explain', 'define', 'how does', 'what are the principles',
        'describe', 'tell me about', 'analogy'
    ]

    # Check for keywords suggesting a need for web search
    if any(keyword in query_lower for keyword in search_keywords):
        use_tavily = True
        reason = "Query contains keywords (e.g., 'latest', 'verify', 'current') indicating a need for recent information or factual lookup."

    # Refine decision for conceptual queries if a search keyword was NOT found
    elif any(phrase in query_lower for phrase in conceptual_phrases):
        use_tavily = False
        reason = "Query appears conceptual and does not require external search. Internal knowledge is likely sufficient."

    # If still False, but query asks for specific entity that might need external context (e.g., specific company name)
    # This is a basic example and could be enhanced with named entity recognition.
    elif len(query_lower.split()) > 4 and not use_tavily: # Heuristic for more specific queries
        use_tavily = False # Default to false for general longer queries unless a specific keyword is hit
        reason = "Query is general and likely answerable from internal knowledge base."

    # If after all checks, it's still not decided, and it's a moderately complex query, default to search
    if not use_tavily and len(query_lower.split()) > 6 and '?' in query_lower:
        use_tavily = True
        reason = "Query is ambiguous or complex; defaulting to web search to ensure comprehensive and up-to-date information."

    # Override if specific 'latest' or 'verify' type words are present, ensuring search
    if 'latest' in query_lower or 'verify' in query_lower or 'current' in query_lower:
        use_tavily = True
        reason = "Query explicitly requests latest/current information or verification, requiring a web search."

    return use_tavily, reason

print("The `should_use_tavily` function is defined and ready for use.")

The `should_use_tavily` function is defined and ready for use.


## Implement ReACT Agent Core

### Subtask:
Create a Python function (e.g., `run_agent`) that serves as the core agent logic. It should take a user query, use `should_use_tavily` to decide on web search, execute `tavily_search` if needed, and then use the OpenAI API to synthesize a final answer. This function must construct a 'Decision Trace' and a 'Tool Trace'.

## Develop ReACT Agent Core

### Subtask:
Build the core ReACT-style agent logic within a function (e.g., `run_agent`). This function will take a user query, interpret intent, call `should_use_tavily` to decide on web search, execute `tavily_search` if needed, and then use the OpenAI API (Responses or Chat Completions) to synthesize a final answer. The agent will construct the 'Decision Trace' (high-level, non-sensitive reasons for search decision) and 'Tool Trace' (search query, top results with title/URL if search was performed).


**Reasoning**:
I need to implement the `run_agent` function as specified in the instructions, which will orchestrate the entire ReACT agent logic, including calling `should_use_tavily`, `tavily_search`, and the OpenAI API, while constructing the 'Decision Trace' and 'Tool Trace'.



In [5]:
import os
import json
from openai import OpenAI

# Ensure should_use_tavily and tavily_search functions are defined or imported here if they were in a separate cell
# For simplicity, assuming they are accessible in the current scope from previous steps.

def run_agent(user_query: str, mode: str = 'Normal Mode') -> tuple[str, str, str]:
    """
    Core ReACT-style agent logic to process a user query, decide on web search, execute it if needed,
    and synthesize a final answer using the OpenAI API.

    Args:
        user_query (str): The user's input query.
        mode (str): The operational mode, either 'Normal Mode' or 'Teach Mode'.

    Returns:
        tuple[str, str, str]: decision_trace, tool_trace, final_answer.
    """
    decision_trace = ""
    tool_trace = ""
    final_answer = ""

    # 1. Decide if web search is needed
    use_tavily, reason = should_use_tavily(user_query)

    # 2. Construct Decision Trace
    decision_trace = f"Search needed: {'Yes' if use_tavily else 'No'}. Reason: {reason}"

    search_results = []
    openai_context = ""

    # 3. Execute web search if needed
    if use_tavily:
        try:
            search_results = tavily_search(user_query)
            if search_results:
                # Format search results for OpenAI context
                formatted_results = []
                for i, result in enumerate(search_results):
                    formatted_results.append(f"Source {i+1}:\nTitle: {result.get('title', 'N/A')}\nURL: {result.get('url', 'N/A')}\nContent: {result.get('content', 'N/A')}")
                openai_context = "\n\n" + "\n\n".join(formatted_results)

                # Construct Tool Trace with top results
                tool_trace_content = []
                for i, result in enumerate(search_results):
                    tool_trace_content.append(f"Title: {result.get('title')}, URL: {result.get('url')}")
                tool_trace = f"Search Query: {user_query}\n\nResults (Top {len(tool_trace_content)}):\n" + "\n".join(tool_trace_content)
            else:
                tool_trace = f"Search Query: {user_query}\n\nNo relevant results found."
        except Exception as e:
            tool_trace = f"Search Query: {user_query}\n\nError during search: {e}"
            openai_context = ""
    else:
        tool_trace = "No external tools used."

    # 4. Define system prompt
    system_prompt = (
        "You are an expert auditor and accountant in the UAE. Your role is to provide helpful, accurate, and clear answers to financial and audit-related questions, adopting a professional tone. "
        "If you use information from web search results, you MUST cite the source (e.g., [1] Title - URL). "
        "Explain complex concepts clearly, using relevant audit or accounting analogies where appropriate to enhance understanding. "
        "Ensure your answers are concise yet comprehensive, addressing all aspects of the user's query. "
        "Always adhere to UAE-specific regulations and practices when applicable."
    )

    # 5. Create messages for OpenAI API
    messages = [
        {"role": "system", "content": system_prompt}
    ]

    user_message_content = f"User Query: {user_query}"
    if openai_context:
        user_message_content += f"\n\nHere are some relevant web search results for context:\n{openai_context}"

    messages.append({"role": "user", "content": user_message_content})

    # 6. Initialize OpenAI client and make API call
    try:
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        response = client.chat.completions.create(
            model="gpt-4o", # Using gpt-4o as specified
            messages=messages,
            temperature=0.7
        )
        final_answer = response.choices[0].message.content
    except Exception as e:
        final_answer = f"An error occurred while generating the response: {e}"
        if not os.environ.get("OPENAI_API_KEY"):
            final_answer += "\nPlease ensure OPENAI_API_KEY is correctly set."

    return decision_trace, tool_trace, final_answer

print("The `run_agent` function is defined and ready.")

The `run_agent` function is defined and ready.


## Build Gradio Interactive Frontend

### Subtask:
Construct the Gradio interface for the agent, including user input, mode selection, clear button, and displaying traces and final answer.


**Reasoning**:
I will implement the Gradio interface as per the instructions, which involves importing the Gradio library, defining a wrapper function to handle mode-based output formatting, creating the Gradio interface with appropriate input/output components, setting up example questions, and launching the application.



In [8]:
import gradio as gr

def run_agent_gradio(user_query: str, mode: str) -> tuple[str, str, str, str, str]:
    """
    Wrapper function to call the core agent logic and format output for Gradio.
    It also handles conditional display of traces based on the selected mode.
    """
    # Call the core agent logic
    decision_trace, tool_trace, final_answer = run_agent(user_query, mode)

    # Prepare outputs for Gradio
    output_decision_trace = decision_trace
    output_tool_trace = tool_trace
    output_final_answer = final_answer
    output_user_question = f"User question: {user_query}"

    # Conditional visibility based on mode
    if mode == 'Normal Mode':
        return output_user_question, output_final_answer, gr.update(value=None, visible=False), gr.update(value=None, visible=False), gr.update(value=output_final_answer, visible=True)
    else: # Teach Mode
        return output_user_question, output_final_answer, gr.update(value=output_decision_trace, visible=True), gr.update(value=output_tool_trace, visible=True), gr.update(value=output_final_answer, visible=True)

# Create Gradio Interface
with gr.Blocks(title="AI Agent with ReACT Paradigm") as demo:
    gr.Markdown("# AI Agent with ReACT Paradigm")
    gr.Markdown("Explore the capabilities of an AI Agent using the ReACT paradigm. Select 'Teach Mode' to see the agent's decision-making process (Decision Trace, Tool Trace) or 'Normal Mode' for just the final answer.")

    with gr.Row():
        query_input = gr.Textbox(label='Enter your question:', placeholder='e.g., What is an AI Agent?', lines=2)
        mode_dropdown = gr.Dropdown(choices=['Normal Mode', 'Teach Mode'], label='Mode', value='Normal Mode')

    run_button = gr.Button('Run Agent')

    user_question_output = gr.Textbox(label='User Question', interactive=False, visible=True)
    decision_trace_output = gr.Textbox(label='Decision Trace', interactive=False, visible=False, lines=3)
    tool_trace_output = gr.Textbox(label='Tool Trace', interactive=False, visible=False, lines=5)
    final_answer_output = gr.Textbox(label='Final Answer', interactive=False, visible=True, lines=10)

    # Initialize ClearButton with components to clear
    clear_button = gr.ClearButton(components=[query_input, user_question_output, decision_trace_output, tool_trace_output, final_answer_output])

    # Define the outputs based on mode
    outputs = [
        user_question_output,
        final_answer_output,
        decision_trace_output,
        tool_trace_output,
        final_answer_output # This is intentionally duplicated to handle the case where only final_answer needs to be updated for visibility
    ]

    # Event handler for the Run button
    run_button.click(
        fn=run_agent_gradio,
        inputs=[query_input, mode_dropdown],
        outputs=outputs
    )

    # Define examples
    gr.Examples(
        examples=[
            ['What is a ReACT agent? Explain with audit analogy.', 'Teach Mode'],
            ['What are the latest UAE VAT updates?', 'Normal Mode'],
            ['Create an audit evidence request list for revenue testing.', 'Normal Mode']
        ],
        inputs=[query_input, mode_dropdown],
        fn=run_agent_gradio,
        outputs=outputs,
        cache_examples=False
    )

# Launch the Gradio app
print("Launching Gradio app...")
demo.launch(share=False)
print("Gradio app launched.")

Launching Gradio app...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Gradio app launched.


## Integrate Agent Output Formatting

### Subtask:
Review and confirm that the agent's output is consistently formatted into distinct sections ('Decision Trace', 'Tool Trace', 'Final Answer') as specified, including the UAE audit/accounting tone and citations for the final answer.


### Review of `run_agent` function - Trace Construction

#### Decision Trace:

The `run_agent` function constructs the `decision_trace` as follows:

```python
decision_trace = f"Search needed: {'Yes' if use_tavily else 'No'}. Reason: {reason}"
```

This format directly matches the requirement: "'Search needed? Yes/No, Why?'".

#### Tool Trace:

If `use_tavily` is `True` and `search_results` are found, the `tool_trace` is constructed as:

```python
tool_trace_content = []
for i, result in enumerate(search_results):
    tool_trace_content.append(f"Title: {result.get('title')}, URL: {result.get('url')}")
tool_trace = f"Search Query: {user_query}\n\nResults (Top {len(tool_trace_content)}):\n" + "\n".join(tool_trace_content)
```

This correctly includes the search query and a list of results with "title, URL".

If `use_tavily` is `True` but no results are found, or an error occurs during search, it handles these cases:

```python
else:
    tool_trace = f"Search Query: {user_query}\n\nNo relevant results found."
```
or
```python
except Exception as e:
    tool_trace = f"Search Query: {user_query}\n\nError during search: {e}"
```

If `use_tavily` is `False`, the `tool_trace` is set to:

```python
else:
    tool_trace = "No external tools used."
```

This construction also aligns with the requirement: "'search query + results (title, URL) if searched, otherwise 'No external tools used'."

**Conclusion:** Both `decision_trace` and `tool_trace` are constructed as per the specified requirements within the `run_agent` function.

### Review of `run_agent` function - Final Answer Formatting

#### System Prompt Analysis:

The `run_agent` function defines the `system_prompt` as follows:

```python
system_prompt = (
    "You are an expert auditor and accountant in the UAE. Your role is to provide helpful, accurate, and clear answers to financial and audit-related questions, adopting a professional tone. "
    "If you use information from web search results, you MUST cite the source (e.g., [1] Title - URL). "
    "Explain complex concepts clearly, using relevant audit or accounting analogies where appropriate to enhance understanding. "
    "Ensure your answers are concise yet comprehensive, addressing all aspects of the user's query. "
    "Always adhere to UAE-specific regulations and practices when applicable."
)
```

This `system_prompt` explicitly addresses all aspects of the `final_answer` formatting requirements:

-   **Helpful, accurate, clear answers**: "Your role is to provide helpful, accurate, and clear answers..."
-   **UAE audit/accounting tone**: "You are an expert auditor and accountant in the UAE... adopting a professional tone... Always adhere to UAE-specific regulations and practices when applicable."
-   **Citations if web search used**: "If you use information from web search results, you MUST cite the source (e.g., [1] Title - URL)."

Additionally, the user message content passed to OpenAI includes the `openai_context` (formatted search results) when `use_tavily` is true, providing the model with the necessary information to generate citations.

**Conclusion:** The `system_prompt` effectively guides the OpenAI model to produce a `final_answer` that meets all specified formatting and content requirements, including the UAE audit/accounting tone and citations.

### Review of Gradio Interface - Output Display and Mode Handling

#### `run_agent_gradio` Function Analysis:

The `run_agent_gradio` function acts as a wrapper, calling the core `run_agent` function and then conditionally formatting its outputs for the Gradio interface:

```python
def run_agent_gradio(user_query: str, mode: str) -> tuple[str, str, str, str, str]:
    # Call the core agent logic
    decision_trace, tool_trace, final_answer = run_agent(user_query, mode)

    # Prepare outputs for Gradio
    output_decision_trace = decision_trace
    output_tool_trace = tool_trace
    output_final_answer = final_answer
    output_user_question = f"User question: {user_query}"

    # Conditional visibility based on mode
    if mode == 'Normal Mode':
        return output_user_question, output_final_answer, gr.update(value=None, visible=False), gr.update(value=None, visible=False), gr.update(value=output_final_answer, visible=True)
    else: # Teach Mode
        return output_user_question, output_final_answer, gr.update(value=output_decision_trace, visible=True), gr.update(value=output_tool_trace, visible=True), gr.update(value=output_final_answer, visible=True)
```

-   **Output Handling**: It correctly captures `decision_trace`, `tool_trace`, and `final_answer` from `run_agent`.
-   **Mode Logic**: It uses an `if/else` block to control the visibility of the trace components based on the `mode` parameter.
    -   In 'Normal Mode', `decision_trace_output` and `tool_trace_output` are set to `visible=False` and their values to `None` using `gr.update`. The `final_answer_output` is set to `visible=True`.
    -   In 'Teach Mode', `decision_trace_output` and `tool_trace_output` are set to `visible=True` with their respective values, and `final_answer_output` is also `visible=True`.

#### Gradio Interface (`demo`) Analysis:

The Gradio interface defines the `gr.Textbox` components for displaying the outputs:

```python
    user_question_output = gr.Textbox(label='User Question', interactive=False, visible=True)
    decision_trace_output = gr.Textbox(label='Decision Trace', interactive=False, visible=False, lines=3)
    tool_trace_output = gr.Textbox(label='Tool Trace', interactive=False, visible=False, lines=5)
    final_answer_output = gr.Textbox(label='Final Answer', interactive=False, visible=True, lines=10)
```

-   **Initial Visibility**: `decision_trace_output` and `tool_trace_output` are initially set to `visible=False`, which aligns with the 'Normal Mode' default.

The `outputs` list passed to the `run_button.click` method correctly maps to these components, ensuring that the return values from `run_agent_gradio` update the intended `gr.Textbox` elements:

```python
    outputs = [
        user_question_output,
        final_answer_output,
        decision_trace_output,
        tool_trace_output,
        final_answer_output # Duplicated to handle updates correctly across modes
    ]

    run_button.click(
        fn=run_agent_gradio,
        inputs=[query_input, mode_dropdown],
        outputs=outputs
    )
```

**Conclusion:** The `run_agent_gradio` function and the Gradio interface are correctly configured to pass and display the `decision_trace`, `tool_trace`, and `final_answer` outputs, adhering to the visibility rules for 'Normal Mode' and 'Teach Mode'. All output formatting requirements are met.

## Add Educational Lab Exercise

### Subtask:
Include a markdown cell with a mini 'lab exercise' after the agent implementation. This exercise will prompt users to modify the `should_use_tavily` gating rules and observe the agent's behavior, encouraging hands-on learning.


## Educational Lab Exercise: Customizing Search Gating Logic

This mini-lab exercise is designed to help you understand and experiment with the core decision-making process of our ReACT agent, specifically how it decides whether to perform a web search using Tavily.

By modifying the `should_use_tavily` function, you can directly influence the agent's behavior and observe the impact on its 'Decision Trace' and 'Final Answer'. This hands-on experience will deepen your understanding of agent design and tool integration.

### Instructions:
1.  **Locate the `should_use_tavily` function**: Scroll up in the notebook and find the code cell where the `should_use_tavily` function is defined.
2.  **Experiment with Gating Rules**:
    *   **Add new `search_keywords`**: Think of specific words or phrases that, when present in a user's query, *always* necessitate a web search. For example, you might add 'price', 'cost', 'review' for product-related queries.
    *   **Add new `conceptual_phrases`**: Identify phrases that indicate a general or theoretical question that likely does *not* require an external search. For instance, 'what is the meaning of', 'describe the concept of'.
    *   **Modify existing conditions**: Adjust the heuristic conditions, such as the `len(query_lower.split()) > 6` rule, to make it more or less sensitive.
    *   **Add entirely new rules**: Consider other factors that might influence the decision, like the presence of numbers or specific entity types (though this might require more advanced NLP).
3.  **Re-run the Code Cells**: After making changes to the `should_use_tavily` function, you must re-run its defining cell (the cell where it is implemented) and then re-run the Gradio app launch cell (`demo.launch(share=False)`) for your changes to take effect.
4.  **Test Your Changes**: Use the Gradio interface to test various queries, both in 'Normal Mode' and especially in 'Teach Mode' to observe the 'Decision Trace'.
    *   **Example Test Queries**:
        *   Queries containing your new `search_keywords` (e.g., "What is the current market price of gold?")
        *   Queries containing your new `conceptual_phrases` (e.g., "Describe the concept of intangible assets.")
        *   Queries that previously triggered a search but now shouldn't, or vice-versa.

Observe how the 'Decision Trace' changes and how the agent's 'Final Answer' adapts based on whether it performs a web search or relies solely on its internal knowledge.

## Final Task

### Subtask:
Provide a summary that confirms the notebook is complete and ready for execution, covering all aspects of the user's request.


## Summary:

### Data Analysis Key Findings

*   **Comprehensive Notebook Introduction and Setup**: The notebook includes a detailed markdown introduction explaining AI agents, the ReACT paradigm, the role of tools (Tavily for web search), and the concept of a 'Decision Trace'. All necessary Python packages (`openai`, `tavily-python`, `gradio`) are installed, and `OPENAI_API_KEY` and `TAVILY_API_KEY` are securely set with robust error handling.
*   **Robust Tavily Search Tool**: A `tavily_search` Python function is implemented to interact with the Tavily API, returning structured search results (title, URL, content) with comprehensive error handling, ensuring reliable web search capabilities.
*   **Intelligent Search Gating Logic**: The `should_use_tavily` Python function effectively determines whether a web search is necessary based on the user's query. It uses keywords (e.g., 'latest', 'verify', 'current') and conceptual phrases to make a boolean decision and provides a clear reason, preventing unnecessary API calls.
*   **Core ReACT Agent Implementation**: The `run_agent` Python function orchestrates the ReACT paradigm. It integrates the `should_use_tavily` and `tavily_search` functions, uses the OpenAI API (specifically `gpt-4o`) to synthesize a final answer, and meticulously constructs a 'Decision Trace' (search decision and reason) and a 'Tool Trace' (search query and top results, or "No external tools used").
*   **Professional Output Formatting**: The agent's final answer generation is guided by a system prompt instructing it to adopt a professional UAE audit/accounting tone, provide helpful and accurate answers, and *must* cite sources (e.g., `[1] Title - URL`) if information from web search results is used. Traces are consistently formatted as specified.
*   **Interactive Gradio Frontend**: A Gradio interface is successfully constructed, featuring a textbox for user input, a 'Mode' dropdown ('Teach Mode' for full traces, 'Normal Mode' for final answer only), and a 'Clear' button. It dynamically displays 'User question', 'Decision Trace', 'Tool Trace', and 'Final Answer', preloaded with example questions, and runs in Colab with `share=False`.
*   **Educational Lab Exercise**: A markdown cell is included, detailing a mini-lab exercise that guides users to modify the `should_use_tavily` gating rules and observe the agent's behavior, fostering hands-on learning and understanding of agent decision-making.

### Insights or Next Steps

*   **Enhanced Customization and Fine-tuning**: The educational lab exercise highlights the critical role of the `should_use_tavily` function. A next step could involve exposing more parameters or pre-defined profiles within the Gradio interface to allow users to easily toggle or customize the search gating logic without direct code modification, catering to different domain-specific needs.
*   **Advanced Tool Integration**: While Tavily is a strong start, expanding the agent's toolkit with other specialized tools (e.g., a PDF parser for internal documents, a financial data API, or a code interpreter) would significantly enhance its capabilities for a wider range of audit and accounting tasks.
